In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

# whatsinthepan.com
2. The user agent is marked as * which allows all user agents can scrape
3. I minimized my scraping by starting small to make sure my code worked before scrapping for all of my recipes

In [2]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele + ' '  
    
    # return string   
    return str1  

In [3]:
#finds the groups of recipes
def find_groups(url):
    results = requests.get(url)
    soup1 = BeautifulSoup(results.text, 'html.parser')
    s1 = soup1.find('div', class_ = 'widget-area recipes-bottom')
    href = []
    for item in s1.findAll('section', class_ = 'widget featured-content featuredpost'):
        href.append(item.p.a['href'])

    return href

In [4]:
#finds recipes within the groups
def find_rec(url):
    results = requests.get(url)
    soup1 = BeautifulSoup(results.text, 'html.parser')
    s1 = soup1.find('main', class_ = 'content')
    href = []
    for val in s1.findAll('article'):
        href.append(val.header.a['href'])
    return href

In [5]:
def get_ingredients(url):
    #querying ingredients
    results2 = requests.get(url)
    soup2 = BeautifulSoup(results2.text, 'html.parser')
    ingredients = soup2.find('ul', class_ ='wprm-recipe-ingredients') #find block of ingredients
    #formatting the html to a list
    ls = []
    name = soup2.title.text
    for items in ingredients.findAll('li'):
        ls.append(items.text)
    return name, ls
   

In [6]:
#adding and formatting the ingredients into the dataframe
def addTo(id, name, ls):
    ids = []
    name_ls = []
    text = []
    quantity = []
    unit = []
    ingredient = []
    for val in ls:
        ids.append(id)
        name_ls.append(name) 
        text.append(val)
        #taking the digits from the text as quantity
        temp = [str(i) for i in val.split() if i.isdigit() or i in fractions] 
        quantity.append(listToString(temp))
        #taking values designated as units from the text as units
        temp2 = [str(i) for i in val.split() if i in measurements] 
        unit.append(listToString(temp2))
        #taking the remaining elements from the text as ingredients
        temp3 = [str(i) for i in val.split() if i not in measurements and not i.isdigit() and i not in fractions]
        ingredient.append(listToString(temp3))

    #adding our new rows to the df
    df = pd.DataFrame(columns =['id', 'name', 'text', 'quantity', 'unit', 'ingredient'])
    df['id'] = ids
    df['recipe'] = name_ls
    df['text'] = text
    df['quantity'] = quantity
    df['unit'] = unit
    df['ingredient'] = ingredient
    return(df)

In [7]:
#creating the new df
recipes_df = pd.DataFrame(columns =['id', 'name', 'text', 'quantity', 'unit', 'ingredient'])

In [8]:
#USED IN DATA CLEANING
measurements = ['cups', 'tablespoons', 'teaspoon', 'cup', 'tablespoon', 'tbsp', 'tsp']
fractions = ['⅓', '½', '1/3', '1/2']

In [9]:
#main landing url from where we are scraping
url = 'https://whatsinthepan.com/recipes/'
ids = 1
#myRec = pd.DataFrame(columns =['id', 'name', 'url'])
recipes = []
for groups in find_groups(url):
    reci = find_rec(groups)
    for rec in reci:
        recipes.append(rec)
recipes #big list of recipes hrefs

#scraping all the hrefs from the list we just created and adding to the dataframe
for rec in recipes:
    name, i = get_ingredients(rec)
    #myRec.append([ids, name, url])
    temp = addTo(ids, name , i)
    ids = ids + 1
    recipes_df = pd.concat([recipes_df, temp])
    
recipes_df

,id,name,text,quantity,unit,ingredient
0,1,White Chocolate Cranberry Cookies (Gluten Free...,2 cups Blanched Almond Flour,2,cups,Blanched Almond Flour
1,1,White Chocolate Cranberry Cookies (Gluten Free...,½ tsp salt,½,tsp,salt
2,1,White Chocolate Cranberry Cookies (Gluten Free...,1 tsp baking soda,1,tsp,baking soda
3,1,White Chocolate Cranberry Cookies (Gluten Free...,1/3 cup butter melted,1/3,cup,butter melted
4,1,White Chocolate Cranberry Cookies (Gluten Free...,1/3 cup maple syrup,1/3,cup,maple syrup
...,...,...,...,...,...,...
2,50,Classic Irish Soda Bread with Whiskey Raisins ...,4 cups all-purpose flour,4,cups,all-purpose flour
3,50,Classic Irish Soda Bread with Whiskey Raisins ...,1 teaspoon baking soda,1,teaspoon,baking soda
4,50,Classic Irish Soda Bread with Whiskey Raisins ...,1 teaspoon salt,1,teaspoon,salt
5,50,Classic Irish Soda Bread with Whiskey Raisins ...,2 tbsp turbinado sugar + more for sprinkling,2,tbsp,turbinado sugar + more for sprinkling


In [39]:
df = pd.DataFrame(columns=['recipeID', 'url'])
df['recipeID'] = [i for i in range(len(recipes))]
df['url'] = recipes
temp = recipes_df.merge(df, left_on='id', right_on='recipeID')

In [40]:
temp = temp.drop(axis=0, columns=['text', 'quantity', 'unit', 'ingredient', 'id'])

In [41]:
temp = temp.drop_duplicates(keep='first')

In [42]:
temp = temp.reset_index(drop=True)

In [61]:
conn = sqlite3.connect('mealplanner.db')
cursor = conn.cursor()
recipes_df.to_sql('myRec', con=conn, if_exists='append', index=False)
conn.commit()

In [59]:
#recipes_df['qty'] = recipes_df['quantity'] + recipes_df['unit']
recipes_df['recipeID'] = recipes_df['id']
recipes_df = recipes_df.drop(columns=['id'])

KeyError: 'id'

In [60]:
recipes_df

,ingredient,qty,recipeID
0,Blanched Almond Flour,2 cups,1
1,salt,½ tsp,1
2,baking soda,1 tsp,1
3,butter melted,1/3 cup,1
4,maple syrup,1/3 cup,1
...,...,...,...
2,all-purpose flour,4 cups,50
3,baking soda,1 teaspoon,50
4,salt,1 teaspoon,50
5,turbinado sugar + more for sprinkling,2 tbsp,50


In [310]:
#df to csv
recipes_df.to_csv('recipes.csv')

In [20]:
len(recipes)

50

In [2]:
#read back csv
df = pd.read_csv('recipes.csv')

In [6]:
#drop index column made when converting to csv
df = df.drop(columns =['Unnamed: 0'])

In [7]:
#counting how many columns have salt or sugar
salt = 0
sugar = 0
for row in df['ingredient']:
    if 'salt' in row :
        salt = salt + 1
    if 'sugar' in row :
        sugar = sugar + 1

In [8]:
salt
#there is more salt by count

259

In [9]:
sugar

119

In [84]:
#counting teaspoons of salt and sugar
salt = 0
sugar = 0
for i, row in df.iterrows():
    if 'salt' in row['ingredient'] :
        if row['quantity'] != 'nan':
            try:
                salt = salt + int(row['quantity'])
            except:
                #the fractions are rounded up to half for simplification
                salt = salt + .5
    if 'sugar' in row['ingredient'] :
        if row['quantity'] != 'nan':
            try:
                sugar = sugar + int(row['quantity'])
            except:
                #the fractions are rounded up to half for simplification
                sugar = sugar + .5

In [85]:
salt

122.5

In [86]:
sugar
#there is more sugar by volume

150.5

# I spent 4 hours on this assignment